![little bot](little_bot.webp)

In [9]:
from typing import List

from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import Message, SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.service import OrchestrationService

YOUR_API_URL = ""


def load_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()
    

## A simple chat bot

In [10]:
config = OrchestrationConfig(
            template=Template(
                messages=[
                    SystemMessage("You are a helpful chatbot assistant."),
                    UserMessage("{{?user_query}}"),
                ],
            ),
            llm=LLM(name="gpt-4o", parameters={"temperature": 0.0}),
        )

class ChatBot:
    def __init__(self, orchestration_service: OrchestrationService, config: OrchestrationConfig):
        self.service = orchestration_service
        self.history: List[Message] = []
        self.config = config

    def chat(self, user_input):
        response = self.service.run(
            config=self.config,
            template_values=[
                TemplateValue(name="user_query", value=user_input),
            ],
            history=self.history,
        )

        message = response.orchestration_result.choices[0].message

        self.history = response.module_results.templating
        self.history.append(message)

        return message.content

service = OrchestrationService(api_url=YOUR_API_URL)
bot = ChatBot(orchestration_service=service, config=config)


In [11]:
print(bot.chat("Hello, how are you?"))

Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


In [12]:
print(bot.chat("What's the weather like today?"))

I don't have real-time capabilities to check the current weather. However, you can easily find the weather forecast by checking a weather website, using a weather app on your phone, or asking a voice-activated assistant like Siri, Google Assistant, or Alexa. How else can I assist you today?


In [13]:
print(bot.chat("Can you remember what I first asked you?"))

Yes, you first asked me, "Hello, how are you?" Is there something specific you'd like to discuss or need help with?


## Making our chat bot responsible
Adding content filtering and data masking by only changing the config

In [14]:
from gen_ai_hub.orchestration.models.data_masking import DataMasking
from gen_ai_hub.orchestration.models.sap_data_privacy_integration import SAPDataPrivacyIntegration, MaskingMethod, \
    ProfileEntity
from gen_ai_hub.orchestration.models.azure_content_filter import AzureContentFilter, AzureThreshold


data_masking = DataMasking(
    providers=[
        SAPDataPrivacyIntegration(
            method=MaskingMethod.ANONYMIZATION,  # or MaskingMethod.PSEUDONYMIZATION
            entities=[
                ProfileEntity.EMAIL,
                ProfileEntity.PHONE,
                ProfileEntity.PERSON,
                ProfileEntity.ORG,
                ProfileEntity.LOCATION
            ]
        )
    ]
)

output_filter = AzureContentFilter(hate=AzureThreshold.ALLOW_SAFE,
                                   violence=AzureThreshold.ALLOW_SAFE_LOW,
                                   self_harm=AzureThreshold.ALLOW_SAFE_LOW_MEDIUM,
                                   sexual=AzureThreshold.ALLOW_ALL)

config.data_masking = data_masking
config.output_filters = [output_filter]


Handling sensitive information

In [15]:
cv = load_text_file("data/cv.txt")
prompt = "Summarize the following CV in 10 sentences: {cv}".format(cv=cv)

print(prompt + "\n")

Summarize the following CV in 10 sentences: Patrick Morgan
+49 30 23125 123
patric.morgan@example.com

Highlights

Strategic and financial planning expert
Accurate forecasting
Process implementation
Staff leadership and development
Business performance improvement
Proficient in SAP, Excel VBA
Education
Master of Science: Finance - 2014
Harvard University, Boston

Bachelor of Science: Finance - 2011
Harvard University, Boston

Certifications
Certified Management Accountant

Summary
Skilled Financial Manager adept at increasing work process efficiency and profitability through functional and technical analysis.
Successful at advising large corporations, small businesses, and individual clients. Areas of expertise include asset allocation, investment strategy, and risk management.

Experience
Finance Manager - 09/2016 to 05/2018
M&K Group, York

Manage the modelling, planning, and execution of all financial processes.
Carry short and long-term custom comprehensive financial strategies to 

In [16]:
print(bot.chat(prompt))

MASKED_PERSON is a skilled Financial Manager with expertise in strategic and financial planning, accurate forecasting, and process implementation. They have a strong background in staff leadership and development, as well as business performance improvement. Proficient in SAP and Excel VBA, they hold a Master of Science in Finance (2014) and a Bachelor of Science in Finance (2011) from MASKED_ORG. They are also a Certified Management Accountant. 

Their professional experience includes managing financial processes, developing comprehensive financial strategies, and recommending innovative revenue-generating alternatives. They have successfully advised large corporations, small businesses, and individual clients on asset allocation, investment strategy, and risk management. In their role as Finance Manager at MASKED_ORG from 2016 to 2018, they executed financial strategies, conducted market research, and negotiated with potential investors. Prior to that, from 2013 to 2016, they drafted